In [2]:
import cnvpytor
import subprocess

In [4]:
filename = "test_with_del_400_1_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip()) + 1
line_count

1129760

In [5]:
command = f"wgsim -d 300 -N {line_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1699905340
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 56487872


b'chr22\t630\tC\tY\t+\nchr22\t951\t-\tCCT\t-\nchr22\t1697\tG\tS\t+\nchr22\t2696\tG\tA\t-\nchr22\t4232\tA\tR\t+\nchr22\t4351\tT\tK\t+\nchr22\t4400\tA\tW\t+\nchr22\t4537\tA\tM\t+\nchr22\t5808\tT\tY\t+\nchr22\t6930\tA\tW\t+\nchr22\t7306\tT\tK\t+\nchr22\t7465\tG\tR\t+\nchr22\t8852\tG\tK\t+\nchr22\t9903\tC\tS\t+\nchr22\t10853\tA\tW\t+\nchr22\t11223\tA\tG\t-\nchr22\t11764\tA\tW\t+\nchr22\t13141\tA\tC\t-\nchr22\t13962\tGTGC\t-\t-\nchr22\t15795\tC\tS\t+\nchr22\t17244\tT\tK\t+\nchr22\t17494\t-\tC\t+\nchr22\t17613\tT\tG\t-\nchr22\t18229\tA\tW\t+\nchr22\t19493\tA\tG\t-\nchr22\t20967\t-\tA\t+\nchr22\t23256\tA\tM\t+\nchr22\t23795\tA\tW\t+\nchr22\t24117\t-\tT\t+\nchr22\t24612\tC\tS\t+\nchr22\t28036\tG\tA\t-\nchr22\t28492\tT\tW\t+\nchr22\t28998\tG\tC\t-\nchr22\t29114\tA\tT\t-\nchr22\t29284\tG\tR\t+\nchr22\t30701\t-\tAG\t-\nchr22\t31022\tC\tA\t-\nchr22\t32347\tA\tG\t-\nchr22\t33353\tC\tY\t+\nchr22\t34871\tC\tY\t+\nchr22\t35431\tC\tT\t-\nchr22\t37799\tA\tW\t+\nchr22\t39138\tT\t-\t-\nchr22\t39261\tA\tR\

In [7]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [8]:
# sam to bam file conversion
!samtools view -bS biasOut.sam > biasOut.bam
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam


[bam_sort_core] merging from 1 files and 1 in-memory blocks...


In [9]:
print(cnvpytor.__version__)
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)
app.rd(["biasOut_sorted.bam"])

1.3.1


In [10]:
calls = app.call([100, 1000, 10000, 100000, 1000000], print_calls=True)

duplication	chr22:18650001-18880000	230000	4.2821	4.518656e-05	3.023060e+01	4.518656e-05	3.023060e+01	0.0002	0.0000	1629600
duplication	chr22:21460001-21680000	220000	1.7736	4.903345e-05	0.000000e+00	4.903345e-05	0.000000e+00	0.0005	0.0000	850700
duplication	chr22:28460001-28600000	140000	5.6981	1.022443e-07	0.000000e+00	1.022443e-07	0.000000e+00	0.0000	0.0000	7850700
duplication	chr22:30800001-30980000	180000	5.9066	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	0.0000	10190700
duplication	chr22:35940001-36200000	260000	3.9645	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	0.0000	14164900
duplication	chr22:44150001-44470000	320000	3.9391	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0001	0.0000	5894900
duplication	chr22:46380001-46680000	300000	2.9630	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0001	0.0000	3684900
